In [1]:
import pandas as pd
import numpy as np
from xbbg import blp

In [2]:
DO_HISTORY = False

***

In [3]:
TICKS = ['CT10', 'CB3', 'CTII10']
TYPE = 'Govt'
bbcodes = [f'{tick} {TYPE}' for tick in TICKS]
FLDS = [
    'NAME',
    'SECURITY_TYP',
    'MATURITY',
    'MTY_YEARS',
    'CPN',
    'INT_ACC',
    'AMT_OUTSTANDING',
    'BID_YIELD',
    'PX_BID',
    'QT_SPEC',
    'ISSUE_DT',
    'LAST_UPDATE_DT',
    'IS_US_ON_THE_RUN',
    'PRINCIPAL_FACTOR',
        ]
info = blp.bdp(bbcodes,FLDS)

In [4]:
def decimal_to_32nds(price: float) -> str:
    """
    Convert a decimal price to a US Treasury note quote in 32nds.

    Parameters
    ----------
    price : float
        Price in decimal (e.g., 100.234375)

    Returns
    -------
    str
        Quote string in the format "AAA-BB" or "AAA-BB+" if there's a half 32nd.

    Examples
    --------
    >>> decimal_to_32nds(100.15625)
    '100-05+'
    >>> decimal_to_32nds(99.96875)
    '99-31+'
    >>> decimal_to_32nds(101.0)
    '101-00'
    """
    # Split into whole number and fractional part
    whole = int(price)
    frac = price - whole

    # Convert fractional part to 64ths and round to nearest
    total_64ths = round(frac * 64)

    # Determine number of whole 32nds and if there is a half 32nd
    n32 = total_64ths // 2
    half_flag = total_64ths % 2

    # Handle rollover if rounding pushes to the next whole point
    if n32 == 32:
        whole += 1
        n32 = 0
        half_flag = 0

    # Build the quote string
    quote = f"{whole}-{n32:02d}"
    if half_flag:
        quote += "+"
    return quote

In [5]:
ticks_32nds = ['CT10 Govt','CTII10 Govt']
ticks_disc = list(set(bbcodes) - set(ticks_32nds))

for tick in bbcodes:
    if tick in ticks_32nds:
        info.loc[tick,'px_bid'] = decimal_to_32nds(info.loc[tick,'px_bid'])
    elif tick in ticks_disc:
        info.loc[tick,'px_bid'] /= 100


info['cpn'] /= 100
info['bid_yield'] /= 100

In [6]:
info.rename(columns={
    'amt_outstanding':'amount issued',
    'security_typ':'type',
    'principal_factor':'inflation adjustment',
    'issue_dt':'issue date',
    'bid_yield':'ytm (bid)',
    'maturity':'maturity date',
    'px_bid':'price quoted (bid)',
    'mty_years':'ttm (years)',
    'int_acc':'accrued interest',
    'cpn':'coupon rate',
    'qt_spec':'quote specification',
    'last_update_dt':'quote date',
    }, inplace=True)

In [7]:
FLDS_EXPORT = [
    'name',
    'quote date',
    'issue date',
    'maturity date',
    'amount issued',
    'ttm (years)',
    'inflation adjustment',
    'coupon rate',
    'accrued interest',
    'price quoted (bid)',
    'ytm (bid)',
    ]
info_export = info[FLDS_EXPORT]

***

In [8]:
if DO_HISTORY:
    DATESTART = '2000-01-01'
    DATEEND = '2025-05-31'

    FLDS = ['LAST_PRICE']
        
    ts = blp.bdh(bbcodes, FLDS, start_date=DATESTART,end_date=DATEEND)


    ts.index = pd.to_datetime(ts.index)
    ts.index.name = 'date'
    ts = ts.xs('LAST_PRICE', axis=1, level=1)
    ts.dropna(inplace=True)

    ts_export = ts.copy()

***

# Export Data

In [11]:
styled = (
    info_export.T.style
        .format('{:%Y-%m-%d}',subset=(['quote date','issue date','maturity date'], slice(None)))
        .format('{:.0e}',subset=(['amount issued'], slice(None)))
        .format('{:.2f}',subset=(['ttm (years)','accrued interest'], slice(None)))
        .format('{:.3%}',subset=(['inflation adjustment','coupon rate','ytm (bid)'], slice(None)))
        .format('{:.3%}',subset=(['price quoted (bid)'], ticks_disc))
)

display(styled)

,CB3 Govt,CT10 Govt,CTII10 Govt
name,TREASURY BILL,US TREASURY N/B,TSY INFL IX N/B
quote date,2025-07-04,2025-07-04,2025-07-04
issue date,2024-10-03,2025-05-15,2025-01-31
maturity date,2025-10-02,2035-05-15,2035-01-15
amount issued,2e+11,1e+11,6e+10
ttm (years),0.24,9.85,9.52
inflation adjustment,100.000%,100.000%,101.691%
coupon rate,0.000%,4.250%,2.125%
accrued interest,0.00,0.61,1.02
price quoted (bid),4.258%,99-06+,101-03


In [10]:
date_quote = np.unique(info_export['quote date'].to_list())[0]

OUTFILE = f'../data/treasury_quote_comparison_{date_quote:{'%Y-%m-%d'}}.xlsx'

with pd.ExcelWriter(OUTFILE) as writer:  
    info_export.T.to_excel(writer, sheet_name= f'treasury specs', index=True)
    if DO_HISTORY:
        ts_export.to_excel(writer, sheet_name= f'treasury price history', index=True)